In [1]:
import numpy as np
import astropy.constants as const
import os
try:
    import makedustopac
except ImportError:
    print('did not import makedustopac')
    pass

# single grain size from Dsharp opac dustopac.inp only 1 input block
# make a flow chart
#

did not import makedustopac


In [40]:
#############
# Constants #
#############
au   = const.au.cgs.value    # Astronomical Unit  [cm]
pc   = const.pc.cgs.value    # Parsec             [cm]
Msun = const.M_sun.cgs.value # Solar Mass         [g]
Lsun = const.L_sun.cgs.value # Solar Luminosity   [ergs/s]
Rsun = const.R_sun.cgs.value # Solar Radius       [cm]
Tsun = 5780                  # Solar Temperature  [K] 
pi   = np.pi                 # circle stuff lol   [radians]
##########################
# radial cell wall setup #
##########################

rin            = 0.1 * au        # inner radius [cm]                      # Disk Param
rout           = 100 * au        # outer radius [cm]                      # Disk Param
r_factor       = 5               # point where to change grid style       # Grid Param
rchange        = r_factor * rin  # radius where grid changes from [au]    # Grid Param
nr_inner_cells = 3               # number of cell for inner FINE region   # Grid Param
nr_outer_cells = 3               # number of cell for outer COARSE region # Grid Param

r_grid_params = [                                             
    [rin_au, rchange, nr_inner_cells + 1, False],      # inner radius args
    [*np.log10([rchange,rout_au]), nr_outer_cells + 1] # outer radius args   
]

# to avoid duplication, endpoint is set to FALSE where rchange 
# will not be accounted for in inner grid but recovered in outer

r_inner   = np.linspace(*r_grid_params[0])     # inner FINE grid style
r_outer   = np.logspace(*r_grid_params[1])     # outer COARSE grid style
r_grid    = np.concatenate([r_inner, r_outer]) # merge the two grids


####################################
# Theta cell wall setup (altitude) #
####################################

theta_min         = 0      # minimum altitude                   # grid param
theta_change      = pi / 3 # loacation where grid style changes # grid param
theta_midplane    = pi / 2 # mid-plane location
ntheta_cells_high = 3      # number of cells near the plane     # grid param
ntheta_cells_low  = 3      # number of cells near the plane     # grid param

theta_grid_params = [
    [theta_min, theta_change, ntheta_cells_high + 1, False], # theta high args
    [theta_change, theta_midplane, ntheta_cells_low + 1]     # theta low args
]

theta_grid_high     = np.linspace(*theta_grid_params[0])                         # COARSE grid spacing
theta_grid_low      = np.linspace(*theta_grid_params[1])                         # FINE grid spacing near the mid-plane
theta_grid_1st_half = np.concatenate([theta_grid_high, theta_grid_low])          # 0 - 90 degress
theta_grid_2nd_half = np.flip(pi - theta_grid_1st_half, 0)[1:]                   # from 90 to 180 degress
theta_grid          = np. concatenate([theta_grid_1st_half,theta_grid_2nd_half]) # 


#################################
# Phi cell wall setup (azimuth) #
#################################

nphi_cells        = 3                                   # number of cells in polar axis # Grid param
phi_grid          = np.linspace(0, 2*pi, nphi_cells + 1) # polar angles

##############################
# Find centers of cell walls #
##############################

nr     = len(r_grid)
nphi   = len(phi_grid)
ntheta = len(theta_grid)

r_center_grid     = 0.5 * ( r_grid_cm[0:nr-1] + r_grid_cm[1:nr] )
phi_center_grid   = 0.5 * ( phi_grid[0:nphi-1] + phi_grid[1:nphi] )
theta_center_grid = 0.5 * ( theta_grid[0:ntheta-1] + theta_grid[1:ntheta] )

nr_center     = len(r_center_grid)
nphi_center   = len(phi_center_grid)
ntheta_center = len(theta_center_grid)

#print(nr, nr_center)
#print(nphi, nphi_center)
#print(ntheta, ntheta_center)

###############################
# Write the spatial grid file #
###############################

def writeSpatialGridFile():
    print('Writing amr_grid.inp ...')
    with open('amr_grid.inp', 'w+') as f:
        # Write formating section
        f.write('1\n')                                                      # iformat
        f.write('0\n')                                                      # AMR grid style  (0=regular grid, no AMR)
        f.write('100\n')                                                    # Coordinate system
        f.write('0\n')                                                      # grid info
        f.write('1 1 1\n')                                                  # include coordinate: r, phi, theta   #
        f.write('{} {} {}\n'.format(nr_center, nphi_center, ntheta_center)) # grid  cell info

        # Write grid values
        for r in r_grid_cm:
            f.write('{} '.format(r))
        f.write('\n')
        for phi in phi_grid:
            f.write('{} '.format(phi))
        f.write('\n')
        for theta in theta_grid:
            f.write('{} '.format(theta))
            
print(phi_center_grid)
print(theta_center_grid)
print(r_center_grid)

[1.04719755 3.14159265 5.23598776]
[0.13089969 0.39269908 0.65449847 0.91629786 1.13446401 1.30899694
 1.48352986 1.65806279 1.83259571 2.00712864 2.2252948  2.48709418
 2.74889357 3.01069296]
[5.74081829e+13 1.69232591e+14 2.81057000e+14 3.92881408e+14
 5.59601110e+14 8.35933023e+14 1.24871807e+15]


In [41]:
###################################################################
# Dust Density Profile
###################################################################

# Constants
rc_au      = 100 
Mdust_Msun = 1e-4
gamma      = 1
H100au     = 10
beta       = 2

def coordTransformation(r, theta):
    # convert spherical radius to cylindrical radius
    r_cylindrical = r * np.sin(theta)
    z_cylindrical = r * np.cos(theta)
    return r_cylindrical , z_cylindrical
     
def volumeDustDensity(r, z, rc_au, Mdust_Msun, gamma, H100au, beta):
    # Calculate the volume dust density
    # inputs: radius [cm], z-component [cm], disk mass [g], gamma "radial
    # profile" index, the scale height at 100 au [cm], beta "flaring
    # parameter" index
    
    rc = rc_au * au 
    Mdust = Mdust_Msun * Msun
    
    def scaleHeight():
        # Calculate the scale height at some radius
        #                     r 
        # H(r) = H_100au (---------)^B
        #                  100 au 
        return H100au * au * ( r / ( 100 * au ) )**beta    
    
    def surfaceDustDensity():
        # calculate surface density and initial surface density (mass dependent) at some radius in cgs
        s0 = Mdust * ( gamma - 2 ) / ( 2 * np.pi * (np.exp( ( rout_au / rc_au )**( 2 - gamma ) ) - np.exp( ( rin_au / rc_au )**( 2 - gamma ) ) ) )      
        return s0 * ( r / rc )**( -gamma ) * np.exp( -( r / rc )**( 2 - gamma ) )
    
    return ( 1 / np.sqrt( 2 * np.pi ) ) * ( surfaceDustDensity() / scaleHeight() ) * np.exp( -0.5 * ( z / scaleHeight() )**2 )


dust_density_grid = np.zeros((nphi_center, ntheta_center, nr_center))

for i in range(nphi_center):
    for j, theta in enumerate(theta_center_grid):
        for k, radius in enumerate(r_center_grid): 
            r_cylindrical_jk , z_cylindrical_jk = coordTransformation(r_center_grid[k], theta_center_grid[j])
            density = volumeDustDensity(r_cylindrical_jk, z_cylindrical, rc_au, Mdust_Msun, gamma, H100au, beta)
            dust_density_grid[i][j,k] = density
#r_cyl, z_cyl = coordTransformation(radius_grid_cm, theta_grid)





In [42]:
print(dust_density_grid)
print(dust_density_grid.flatten())

[[[-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -0.00000000e+000 -0.00000000e+000]
  [-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -0.00000000e+000 -0.00000000e+000]
  [-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -0.00000000e+000 -8.68453547e-213]
  [-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -0.00000000e+000 -2.81108326e-065]
  [-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -7.40546691e-216 -4.34867913e-033]
  [-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -2.49066446e-164 -7.96948095e-023]
  [-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -1.00134509e-143 -9.94849920e-019]
  [-0.00000000e+000 -0.00000000e+000 -0.00000000e+000 -0.00000000e+000
   -0.00000000e+000 -1.00134509e-143 -9.9484992

In [53]:
#dim_array = [nphi, ntheta, nr]
dim_array = [4, 4, 4]
n_phi = 5
r = np.arange(1,111,10).reshape(1,-1)
theta = np.linspace(np.pi/2, np.pi,10).reshape(-1,1)
r_cyn = r * np.sin(theta)
print(r)
print(theta)
print(r_cyn, r_cyn.shape)




"""dDG = np.random.randn(*dim_array)

print(dDG)
print(dDG.shape)
dDG = dDG.flatten()
print(dDG)
print(dDG.shape)"""

[[  1  11  21  31  41  51  61  71  81  91 101]]
[[1.57079633]
 [1.74532925]
 [1.91986218]
 [2.0943951 ]
 [2.26892803]
 [2.44346095]
 [2.61799388]
 [2.7925268 ]
 [2.96705973]
 [3.14159265]]
[[1.00000000e+00 1.10000000e+01 2.10000000e+01 3.10000000e+01
  4.10000000e+01 5.10000000e+01 6.10000000e+01 7.10000000e+01
  8.10000000e+01 9.10000000e+01 1.01000000e+02]
 [9.84807753e-01 1.08328853e+01 2.06809628e+01 3.05290403e+01
  4.03771179e+01 5.02251954e+01 6.00732729e+01 6.99213505e+01
  7.97694280e+01 8.96175055e+01 9.94655831e+01]
 [9.39692621e-01 1.03366188e+01 1.97335450e+01 2.91304712e+01
  3.85273975e+01 4.79243237e+01 5.73212499e+01 6.67181761e+01
  7.61151023e+01 8.55120285e+01 9.49089547e+01]
 [8.66025404e-01 9.52627944e+00 1.81865335e+01 2.68467875e+01
  3.55070416e+01 4.41672956e+01 5.28275496e+01 6.14878037e+01
  7.01480577e+01 7.88083117e+01 8.74685658e+01]
 [7.66044443e-01 8.42648887e+00 1.60869333e+01 2.37473777e+01
  3.14078222e+01 3.90682666e+01 4.67287110e+01 5.43891555e+01

'dDG = np.random.randn(*dim_array)\n\nprint(dDG)\nprint(dDG.shape)\ndDG = dDG.flatten()\nprint(dDG)\nprint(dDG.shape)'

In [54]:
count = 0.
for i in range(4):
    for j in range(4):
        for k in range(4):
            count += 1
print (count)

64.0


In [55]:
test_arr = np.random.randn((*dim_array))
test_arr

array([[[-1.56236380e+00,  8.91257839e-01, -6.22286929e-01,
          5.74726757e-01],
        [ 4.68906199e-01, -1.64499136e-01,  1.64999028e-01,
         -1.30417292e+00],
        [ 9.13676972e-01, -5.25458754e-01,  8.85292132e-01,
         -5.74812157e-01],
        [-5.85320644e-01, -7.21903378e-01,  1.12723681e+00,
          1.31216079e-01]],

       [[-4.74000163e-01, -1.83417598e+00,  7.51827050e-02,
         -4.42338910e-01],
        [-6.33906962e-01, -2.40503027e-01,  1.70473139e+00,
          2.62753831e-01],
        [ 4.99682888e-01, -2.93994186e-01, -1.46584855e-01,
          2.96651813e-02],
        [-2.00367708e+00, -4.37794188e-02, -1.38481304e+00,
         -9.25769449e-01]],

       [[-7.51971912e-01,  5.41643024e-01, -2.84072193e-01,
         -1.73605626e-01],
        [-9.44300896e-01,  1.58175030e+00, -5.59727284e-01,
         -1.83521008e+00],
        [ 2.53765013e-01,  7.16124283e-01,  1.34975251e+00,
          2.24964010e+00],
        [-5.22983211e-01,  3.65489995e-

In [63]:
test_arr[0]

array([[-1.5623638 ,  0.89125784, -0.62228693,  0.57472676],
       [ 0.4689062 , -0.16449914,  0.16499903, -1.30417292],
       [ 0.91367697, -0.52545875,  0.88529213, -0.57481216],
       [-0.58532064, -0.72190338,  1.12723681,  0.13121608]])

In [67]:
test_arr[0][0,1] # -> test_arr[phi][theta,r]

0.8912578385766119

In [61]:
test_arr_1d = test_arr.flatten()
print(test_arr_1d.shape)

(64,)


In [70]:
np.ones(16).reshape(2,4,2)

array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]]])

In [73]:
index = np.where(test_arr > 1)
print(index)
test_arr[index]

(array([0, 1, 2, 2, 2, 3], dtype=int64), array([3, 1, 1, 2, 2, 1], dtype=int64), array([2, 2, 1, 2, 3, 3], dtype=int64))


array([1.12723681, 1.70473139, 1.5817503 , 1.34975251, 2.2496401 ,
       1.12935775])

In [74]:
# set up for loop
# run for loop from nick code
# make vectorize versions
# density = (nphi, ntheta, nr)
# density[nphi_i][ntheta_j, nr_k] is that block that represents the phi iteration;
# phi does no calculations all 
# compute the 2d array -> append to density 3d arry collect all theta 1 values

# all theta one values composed into phi block
np.log10([1,10])

array([0., 1.])